In [ ]:
import os
import cv2
import numpy as np
import torch
import dlib
from imutils import face_utils
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from torchvision import transforms  
import torch.nn as nn
from torch.utils.data import Dataset
from albumentations.pytorch import ToTensorV2
import albumentations as A
import torch.nn.functional as F
import math
import time
from imutils import face_utils
from scipy.spatial import distance as dist
import matplotlib.pyplot as plt
import mediapipe
import sys
sys.path.append("../LuminEye-Iris-Center-Localization/")
from BaseModels.resnetModels import BB_model
from BaseModels.efficientnetModels import BB_model
import pandas as pd
from scipy.spatial import distance
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from scipy.spatial import distance

np.random.seed(42)

In [ ]:
# BioID
val_csv_path = "/home/nipun/Documents/Uni_Malta/Datasets/CenterRegression/MP2GAZE/AllCoordinatesMp2GazeTest.csv"


val_df = pd.read_csv(val_csv_path)

In [ ]:
def cropeyes(image, inner_array, outer_array):

    arr = {"top_left": [inner_array[0]-5, inner_array[1]-20],
           "bottom_right": [outer_array[0]+5, outer_array[1]+20]}

    return image[arr["top_left"][1]:arr["bottom_right"][1], arr["top_left"][0]:arr["bottom_right"][0]], arr

In [ ]:
def scaleCoorinatesToOriginalImage(pred_coords,eye_margin):
    
    # {'top_left': array([385, 214]), 'bottom_right': array([426, 226])}
    
    x1 = eye_margin["top_left"][0] + pred_coords[0]
    y1 = eye_margin["top_left"][1] + pred_coords[1]
    
    return [x1,y1]

In [ ]:
def calculateEuclideanDistance(coord1,coord2):
    return (((coord1[0])-float(coord2[0]))^2  + (float(coord1[0])-float(coord2[0]))^2) ^ 0.5

In [ ]:
IMG_DIR = "/home/nipun/Documents/Uni_Malta/Datasets/"

#histogramPath

histogramPath = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-MainPipeLine/BayesClassifier/MatLab/ScleraSkinIris.txt"

In [50]:
def BayesEstimation(img,histogramBinValuesPath,thresh=2.0):
    
    r_img = img[:,:,::-1][:,:,0]
    
    imgGauss = cv2.GaussianBlur(r_img,(3,3),1)
    
    with open(histogramBinValuesPath,"r") as f:
        hist = [float(j.rstrip()) for j in f.readlines()]
        
    imgR_bin = np.zeros((r_img.shape[0],r_img.shape[1])).astype(np.uint8)
    
    
    
    for row in range(r_img.shape[0]):
    
        for column in range(r_img.shape[1]):
            
            binNo = (imgGauss[row,column]) // 4
            
            
            # print(binNo)
            likelihood = hist[binNo]
            
            
            if likelihood >= thresh:
                imgR_bin[row,column] = 1
                
                
    # plt.imshow(imgR_bin)
    # plt.show()
    
    
    
    analysis = cv2.connectedComponentsWithStats(imgR_bin,
                                           4,
                                            cv2.CV_32S)
    (totalLabels, label_ids, values, centroid) = analysis
    
    
    return centroid[-1]

In [51]:
maximizedNormalizedError = []

count = 0
for i, (_, rows) in enumerate(val_df.iterrows()):

        bbox = [int(float(x)) for x in rows["Coordinates"][1:-1].split(',')]

        
        left_inner = bbox[0:2]
        left_center = bbox[2:4]
        left_outer = bbox[4:6]

        right_inner = bbox[6:8]
        right_center = bbox[8:10]
        right_outer = bbox[10:12]
        

        img_path = os.path.join(IMG_DIR, rows["ImageName"][:-1])
        # #
        img = cv2.imread(img_path)


        IPD = distance.euclidean(left_center, right_center)
        

       
       
        cv2.circle(img, (int(left_center[0]), int(
            left_center[1])), 1, (0, 0, 255), -1)

        cv2.circle(img, (int(right_center[0]), int(
            right_center[1])), 1, (0, 0, 255), -1)

        left_eye, Leye = cropeyes(img, left_inner, left_outer)
        right_eye, Reye = cropeyes(img, right_inner, right_outer)
        
        
        left_centroid = BayesEstimation(left_eye, histogramPath)
        right_centroid = BayesEstimation(right_eye, histogramPath)

        cv2.circle(left_eye, (int(left_centroid[0]), int(
            left_centroid[1])), 1, (0, 255, 0), -1)

        cv2.circle(right_eye, (int(right_centroid[0]), int(
            right_centroid[1])), 1, (0, 255, 0), -1)

        pred_l_eye_toOriginaImage = scaleCoorinatesToOriginalImage(
            left_centroid, Leye)
        pred_r_eye_toOriginaImage = scaleCoorinatesToOriginalImage(
            right_centroid, Reye)

        cv2.circle(img, (int(pred_l_eye_toOriginaImage[0]), int(
            pred_l_eye_toOriginaImage[1])), 1, (0, 255, 0), -1)

        cv2.circle(img, (int(pred_r_eye_toOriginaImage[0]), int(
            pred_r_eye_toOriginaImage[1])), 1, (0, 255, 0), -1)

        

        left_eye_euclidea_distance = distance.euclidean(
            pred_l_eye_toOriginaImage, left_center)

        right_eye_euclidea_distance = distance.euclidean(
            pred_r_eye_toOriginaImage, right_center)

        eMax = max(left_eye_euclidea_distance, right_eye_euclidea_distance)/IPD

        print(eMax)

        maximizedNormalizedError.append(eMax)

        # fig, axes = plt.subplots(1, 3)

        # fig.set_figwidth(15)
        # fig.set_figheight(15)

        # axes[0].imshow(img[:, :, ::-1])
        # axes[1].imshow(left_eye[:, :, ::-1])
        # axes[2].imshow(right_eye[:, :, ::-1])

        # axes[0].axis("off")
        # axes[1].axis("off")
        # axes[2].axis("off")

        # plt.tight_layout()
        # plt.show()

        # count += 1

        # if count == 100:
        #     break

        
        
        
    
        # break

0.059381390178548925
0.14740050843919475
0.0569178083975493
0.07342534224949156
0.1047234409598916
0.13165224824108726
0.12449248673898779
0.0712979259548608
0.026926845616903464
0.1850583025494013
0.024613609621415768
0.022656971588023957
0.14340053043087095
0.14730425908090874
0.04104465071862314
0.055094263333023155
0.17048064614539843
0.1466978816161841
0.015302438293496201
0.07248573420831345
0.007572212897393248
0.18879938154265058
0.1787309268878873
0.02312613855610231
0.17483686319112524
0.2174921541622603
0.05128438562674066
0.0438929977231292
0.22156842249700104
0.1276689793133858
0.18237732333442636
0.04026108276490835
0.07647535155432245
0.015967332320925547
0.13568558550268917
0.03780309759112948
0.1463781051745704
0.16357340014051125
0.08218985295343802
0.1715009723556428
0.2165474310478682
0.1669342321872032
0.15877918791986367
0.007632906743951439
0.1979625954295134
0.07495564599425111
0.1882984826999562
0.014165750349266457
0.031071178392446865
0.05124023225001696
0.07

In [52]:
def CheckForLess(list1, val):
    
    l1 = []
    for x in list1:
        
        if x <= val:
            l1.append(x)
            
       
    return l1

In [53]:
e_0_25 = CheckForLess(maximizedNormalizedError,0.25)

e_0_05 = CheckForLess(maximizedNormalizedError,0.05)

e_0_1 = CheckForLess(maximizedNormalizedError,0.1)

In [54]:
(len(e_0_25)/len(maximizedNormalizedError)) *100

90.625

In [55]:
(len(e_0_05)/len(maximizedNormalizedError)) * 100

29.6875

In [56]:
len(e_0_1)/len((maximizedNormalizedError)) * 100

46.09375